<a href="https://colab.research.google.com/github/lorenafc/MGI/blob/main/Statistics_AuthorsCities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import geopandas as gpd
import numpy as np


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# upload the file from your computer
from google.colab import files
uploaded=files.upload()

Saving merged_additional_authors_encoded.xlsx to merged_additional_authors_encoded.xlsx


In [5]:
# Assuming the uploaded file is named 'additional_authors.xlsx'
file_name = 'merged_additional_authors_encoded.xlsx'

# Read the Excel file
authors_small_cities = pd.read_excel(file_name, engine='openpyxl')

authors_small_cities_copy = authors_small_cities

In [6]:
authors_small_cities_copy.rename(columns={'Unnamed: 0': 'indexauthor'}, inplace=True)
authors_small_cities_copy.head()

,indexauthor,starturl,bitrhyear,deathyear,nameandbirthdeathyear,georeferenceurl,borncity,deathcity,activecity,latitude_born,longitude_born,latitude_death,longitude_death,latitude_active,longitude_active
0,0,http://viaf.org/viaf/81048146,1570,1634.0,"Carneiro, Tomé Tavares 1570-1634",viafprocessedfile,,NaN,Rio Tinto,NaN,NaN,NaN,NaN,-6.803828,-35.077569
1,1,http://viaf.org/viaf/180581759,NaN,1775.0,"Kolíček, Caesarij, -1775",viafprocessedfile,,Cieszin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,http://viaf.org/viaf/100161519,NaN,1570.0,"Hertel, Jakob 15..?-1570",http://catalogue.bnf.fr/ark:/12148/cb15011774s,(Chur,NaN,NaN,46.854747,9.526490,NaN,NaN,NaN,NaN
3,3,http://viaf.org/viaf/169350346,1699,1770.0,"Ferro Caaveyro, Lucas Antonio ca. 1699-1770",www.wikidata.org/wiki/Q3393623#sitelinks-wikip...,A Capela,NaN,NaN,43.442068,-8.043400,NaN,NaN,NaN,NaN
4,4,http://viaf.org/viaf/67541728,1502,1600.0,"Aparicio, Sebastián de 1502-1600 Santo",www.wikidata.org/wiki/Q2486114#sitelinks-wikip...,A Gudiña,Puebla City,NaN,42.060548,-7.140352,19.04372,-98.198149,NaN,NaN


In [16]:
# Count the occurrences of each city in the 'activecity' column
city_counts = authors_small_cities_copy['activecity'].value_counts()

# Map the counts to the 'activecity' column
authors_small_cities_copy['activecitycount'] = authors_small_cities_copy['activecity'].map(city_counts)

print(authors_small_cities_copy.head())

   indexauthor                        starturl bitrhyear  deathyear  \
0            0   http://viaf.org/viaf/81048146      1570     1634.0   
1            1  http://viaf.org/viaf/180581759       NaN     1775.0   
2            2  http://viaf.org/viaf/100161519       NaN     1570.0   
3            3  http://viaf.org/viaf/169350346      1699     1770.0   
4            4   http://viaf.org/viaf/67541728      1502     1600.0   

                         nameandbirthdeathyear  \
0             Carneiro, Tomé Tavares 1570-1634   
1                     Kolíček, Caesarij, -1775   
2                     Hertel, Jakob 15..?-1570   
3  Ferro Caaveyro, Lucas Antonio ca. 1699-1770   
4       Aparicio, Sebastián de 1502-1600 Santo   

                                     georeferenceurl  borncity    deathcity  \
0                                  viafprocessedfile                    NaN   
1                                  viafprocessedfile                Cieszin   
2     http://catalogue.bnf.fr/ark:/

In [28]:
### UNIQUE CITIES

# Assuming 'authors_small_cities_copy' is your original DataFrame
# First, create a list of all cities and their latitudes and longitudes
all_cities = authors_small_cities_copy['borncity'].tolist() + authors_small_cities_copy['deathcity'].tolist() + authors_small_cities_copy['activecity'].tolist()
latitudes = authors_small_cities_copy['latitude_born'].tolist() + authors_small_cities_copy['latitude_death'].tolist() + authors_small_cities_copy['latitude_active'].tolist()
longitudes = authors_small_cities_copy['longitude_born'].tolist() + authors_small_cities_copy['longitude_death'].tolist() + authors_small_cities_copy['longitude_active'].tolist()

# Create a DataFrame from the lists
cities_df = pd.DataFrame({
    'city': all_cities,
    'latitude': latitudes,
    'longitude': longitudes
})

# Drop NaN values
# cities_df = cities_df.dropna(subset=['city', 'latitude', 'longitude'])

# Group by 'city' and take the first non-null latitude and longitude for each city
uniquecities_df = cities_df.groupby('city').first().reset_index()

# Now 'cities_df' contains unique cities with their corresponding latitudes and longitudes
print(uniquecities_df.head(15))


                    city   latitude  longitude
0                               NaN        NaN
1            "San Vicenz        NaN        NaN
2          's-Gravendeel  51.780661   4.619875
3          's-Gravenmoer  51.657888   4.940712
4         's-Gravenwezel  51.262486   4.555604
5         's-Gravenzande  52.002829   4.160376
6          's-Heerenberg  51.877945   6.254836
7   'sHeerenberga Gelrus        NaN        NaN
8     (Aire-sur-la-Lys).        NaN        NaN
9               (Anisola        NaN        NaN
10             (Auvergne  45.710013   3.153180
11             (Aveyron)  44.315857   2.506570
12   (Bec-Hellouin Eure)  49.231241   0.721018
13              (Benzing  49.008656  10.844506
14                 (Chur  46.854747   9.526490


In [29]:
uniquecities_df.shape

(44547, 3)

In [30]:
# Calculate the total number of rows in the dataframe with UNIQUE cities
total_rows = len(uniquecities_df)

#number of cities that were not geocoded:
nan_count = uniquecities_df['latitude'].isna().sum()


# Calculate the percentage of NaN values
nan_percentage = (nan_count / total_rows) * 100

print(f"The number of values in the 'latitude' column is: {total_rows}")

print(f"The number of cities that were not geocoded is (NaN values in the 'latitude' column): {nan_count}")

print(f"The percentage of non geocoded cities in total is (NaN values in the 'latitude' column): {nan_percentage:.2f}%")


The number of values in the 'latitude' column is: 44547
The number of cities that were not geocoded is (NaN values in the 'latitude' column): 9292
The percentage of non geocoded cities in total is (NaN values in the 'latitude' column): 20.86%


In [7]:
# # counting unique values born
# n = len(pd.unique(authors_small_cities_copy['borncity']))
# d = len(pd.unique(authors_small_cities_copy['deathcity']))
# a = len(pd.unique(authors_small_cities_copy['activecity']))

# print("No.of.unique values born :", n)

# print("No.of.unique values death :", d)

# print("No.of.unique values active :", a)

No.of.unique values born : 15288
No.of.unique values death : 11965
No.of.unique values active : 26123


In [8]:
# Set the index to the column that you want to keep (e.g., 'your_index_column')
# Then stack the specific columns and reset the index
stacked_df = authors_small_cities_copy.set_index('nameandbirthdeathyear')[['borncity', 'deathcity', 'activecity']].stack().reset_index(name='city')

# Rename the columns as needed
stacked_df.columns = ['nameandbirthdeathyear', 'variable', 'city']

print(stacked_df.head())


unique = len(pd.unique(stacked_df['city']))

print("No.of.unique values total :", unique)


uniquecities = stacked_df['city'].unique()

# Create a new DataFrame with unique city names
unique_cities_df = pd.DataFrame(uniquecities, columns=['city'])

# Now 'unique_cities_df' contains each city name only once
print(unique_cities_df)

unique_cities_df.shape

              nameandbirthdeathyear    variable       city
0  Carneiro, Tomé Tavares 1570-1634    borncity           
1  Carneiro, Tomé Tavares 1570-1634  activecity  Rio Tinto
2          Kolíček, Caesarij, -1775    borncity           
3          Kolíček, Caesarij, -1775   deathcity    Cieszin
4          Hertel, Jakob 15..?-1570    borncity      (Chur


the 20k first records